In [1]:
from xlrd import open_workbook
import urllib
from xml.dom import minidom
import xml.etree.ElementTree as ET
import pandas as pd
from pandas import DataFrame
import json

def getXMLdata(file_path):
    sentence_id_list = []
    sentence_text_list = []
    f = (file_path)
    tree = ET.parse(f)
    doc = tree.getroot()    

    for child_level1 in doc:
        for child_level2 in child_level1:
            sentence_id = child_level2.attrib["sid"]
            sentence_text = child_level2.text
            sentence_id_list.append(sentence_id)
            sentence_text_list.append(sentence_text)
    
    d = {'sentence_id':sentence_id_list,'sentence_text':sentence_text_list}
    xml_data = pd.DataFrame(d, columns = ["sentence_id","sentence_text"])
    return xml_data   
            
def getannotation(file_path):
    citance_number_list = []
    reference_article_list = []
    citing_article_list = []
    citation_text_list = []
    reference_text_list = []
    discourse_facet_list = []
    annotator_list = []
    
    f = open(file_path, "r")
    for line in f.readlines():
        if line != "\n":
            components = line.split(" | ")
            citance_number = components[0].lstrip().strip().replace("Citance Number: ","")
            reference_article = components[1].lstrip().strip().replace("Reference Article: ","")
            citing_article = components[2].lstrip().strip().replace("Citing Article: ","")
            citation_text = components[6].lstrip().strip().replace("Citation Text: ","")
            reference_text = components[8].lstrip().strip().replace("Reference Text: ","")
            discourse_facet = components[9].lstrip().strip().replace("Discourse Facet: ","")
            annotator = components[10].lstrip().strip().replace("Annotator: ","").replace(" |","")
            
            
            citance_number_list.append(citance_number)
            reference_article_list.append(reference_article)
            citing_article_list.append(citing_article)
            citation_text_list.append(citation_text)
            reference_text_list.append(reference_text)
            discourse_facet_list.append(discourse_facet)
            annotator_list.append(annotator)
            
    d = {'citance_number':citance_number_list,'reference_article':reference_article_list,
         'citing_article':citing_article_list,'citance_text':citation_text_list,
        'reference_text':reference_text_list,'discourse_facet':discourse_facet_list,'annotator':annotator_list}
    
    annotation_data = pd.DataFrame(d, columns = ['citance_number','reference_article',
                                                 'citing_article','citance_text',
                                                'reference_text','discourse_facet','annotator'])
    return annotation_data        
            
def getjsondata(file_path):
    with open(file_path) as f:
        data = json.load(f)
        for items in data:
            citation_no = items["citance_No"]
            citing_paper_id = items["citing_paper_id"]
            citing_paper_author = items["citing_paper_authors"]
            citance_text_raw = items["raw_text"]
            citance_text_clean = items["clean_text"]
    

In [2]:
import os

training_data_2018_path = "/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018"
training_data_2019_path = "/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019"

#2018 data
reference_papers_2018 = [dI for dI in os.listdir(training_data_2018_path) if os.path.isdir(os.path.join(training_data_2018_path,dI))]
print ("Number of reference papers in 2018 dataset: ", len(reference_papers_2018))
# print (reference_papers_2018)
    
#2018 data preprocessing
all_2018_data = pd.DataFrame(columns = ['citance_number','reference_article',
                                                 'citing_article','citance_text',
                                                'reference_text','discourse_facet','annotator'])
for reference_paper in reference_papers_2018:
    reference_paper_folder_path = "/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/" + str(reference_paper)
    reference_paper_subfolders = os.walk(reference_paper_folder_path)
    for subfolder in reference_paper_subfolders:
        if "annotation" in subfolder[0]:
            annotation_file = os.listdir(subfolder[0])
            annotation_file_path = subfolder[0] + "/" + annotation_file[0]
            print (annotation_file_path)            
            training_annotation_data = getannotation(annotation_file_path)
            all_2018_data = pd.concat([all_2018_data,training_annotation_data])            
all_2018_data.to_csv("/home/lhoang2/CLSciSumm_2020/all_training_annotation_2018.csv")



#2019 data
reference_papers_2019 = [dI for dI in os.listdir(training_data_2019_path) if os.path.isdir(os.path.join(training_data_2019_path,dI))]
print ("Number of reference papers in 2019 dataset: ", len(reference_papers_2019))
# print (reference_papers_2019)

#2019 data preprocessing
all_2019_data = pd.DataFrame(columns = ['citance_number','reference_article',
                                                 'citing_article','citance_text',
                                              'reference_text','discourse_facet','annotator'])
for reference_paper in reference_papers_2019:
    reference_paper_folder_path = "/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/" + str(reference_paper)
    reference_paper_subfolders = os.walk(reference_paper_folder_path)
    for subfolder in reference_paper_subfolders:
        if "annotation" in subfolder[0]:
            annotation_file = os.listdir(subfolder[0])
            annotation_file_path = subfolder[0] + "/" + annotation_file[0]
            print (annotation_file_path)            
            training_annotation_data = getannotation(annotation_file_path)
            all_2019_data = pd.concat([all_2019_data,training_annotation_data])
            
all_2019_data.to_csv("/home/lhoang2/CLSciSumm_2020/all_training_annotation_2019.csv")

Number of reference papers in 2018 dataset:  40
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/D10-1058/annotation/D10-1058.annv3.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/D09-1023/annotation/D09-1023.annv3.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/N06-2049/annotation/N06-2049.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/P98-1081/annotation/P98-1081.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/E03-1020/annotation/E03-1020.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/C00-2123/annotation/C00-2123.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/P98-2143/annotation/P98-2143.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-Training-Set-2018/W08-2222/annotation/W08-2

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/E06-1043/annotation/E06-1043.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/D07-1061/annotation/D07-1061.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P91-1017/annotation/P91-1017.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W11-1801/annotation/W11-1801.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P07-1092/annotation/P07-1092.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W02-1018/annotation/W02-1018.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J04-2003/annotation/J04-2003.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P05-1067/annotation/P05-1067.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W06-1607/annotation/W06-1607.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P04-3022/annotation/P04-3022.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/H93-1051/annotation/H93-1051.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P06-1121/annotation/P06-1121.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J91-4003/annotation/J91-4003.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/D08-1068/annotation/D08-1068.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J96-1001/annotation/J96-1001.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P08-1101/annotation/P08-1101.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W93-0301/annotation/W93-0301.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P05-1010/annotation/P05-1010.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J92-1001/annotation/J92-1001.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/D07-1074/annotation/D07-1074.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/E87-1002/annotation/E87-1002.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/N10-1061/annotation/N10-1061.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P02-1040/annotation/P02-1040.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J02-1002/annotation/J02-1002.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/D08-1035/annotation/D08-1035.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/N03-1021/annotation/N03-1021.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J99-4005/annotation/J99-4005.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/E06-1027/annotation/E06-1027.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W03-1028/annotation/W03-1028.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/A97-1030/annotation/A97-1030.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J03-3005/annotation/J03-3005.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P97-1013/annotation/P97-1013.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

...

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/H05-1011/annotation/H05-1011.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P05-3026/annotation/P05-3026.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P09-1042/annotation/P09-1042.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W03-1014/annotation/W03-1014.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/D07-1031/annotation/D07-1031.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J99-1003/annotation/J99-1003.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W12-3102/annotation/W12-3102.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P04-1056/annotation/P04-1056.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P07-1059/annotation/P07-1059.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/N04-1030/annotation/N04-1030.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/A00-1031/annotation/A00-1031.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W04-0308/annotation/W04-0308.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/N01-1024/annotation/N01-1024.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/E03-1076/annotation/E03-1076.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P96-1006/annotation/P96-1006.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/H05-1044/annotation/H05-1044.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P05-1044/annotation/P05-1044.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P05-1071/annotation/P05-1071.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/D07-1101/annotation/D07-1101.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/E06-1002/annotation/E06-1002.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W97-0322/annotation/W97-0322.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P03-1019/annotation/P03-1019.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/W10-0204/annotation/W10-0204.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/H05-1079/annotation/H05-1079.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P11-1016/annotation/P11-1016.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/N03-1017/annotation/N03-1017.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P00-1027/annotation/P00-1027.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/C96-2141/annotation/C96-2141.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P98-1035/annotation/P98-1035.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/N04-1043/annotation/N04-1043.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P97-1009/annotation/P97-1009.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P10-1044/annotation/P10-1044.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/N09-1037/annotation/N09-1037.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/J03-3002/annotation/J03-3002.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P07-1073/annotation/P07-1073.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P99-1069/annotation/P99-1069.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P08-1108/annotation/P08-1108.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P07-2045/annotation/P07-2045.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/H94-1048/annotation/H94-1048.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/P07-1037/annotation/P07-1037.ann.txt
/efs/CL-SciSumm/scisumm-corpus/data/Training-Set-2019/Ta

In [3]:
import re

#2018 data
print ("2018 DATA ANALYSIS: ")
citing_text_2018 = all_2018_data["citance_text"]
total_citances = len(citing_text_2018)
total_number_citing_sentences = 0
count_multiple_citing_sentences = 0
for citing_text in citing_text_2018:
#     text_only = re.search('>(.*)</S', citing_text)
#     print (text_only.group(1) + "\n")
#     print (citing_text + "\n")
    number_citing_sentences = citing_text.count("</S>")
    if number_citing_sentences >=2:
        count_multiple_citing_sentences += 1
    total_number_citing_sentences = total_number_citing_sentences + number_citing_sentences
print ("Number of citances: ", total_citances)
print ("Number of sentences from citances: ",total_number_citing_sentences)
print ("Average number of sentences per citance: ", total_number_citing_sentences/total_citances)
print ("Number of citances that have more than 1 sentence: ", count_multiple_citing_sentences)
    
reference_text_2018 = all_2018_data["reference_text"]
total_references = len(reference_text_2018)
total_number_reference_sentences = 0
count_multiple_references_sentences = 0
for reference_text in reference_text_2018:
#     text_only = re.search('>(.*)</S', citing_text)
#     print (text_only.group(1) + "\n")
#     print (citing_text + "\n")
    number_reference_sentences = reference_text.count("</S>")
    if (number_reference_sentences>=2):
        count_multiple_references_sentences += 1
    total_number_reference_sentences = total_number_reference_sentences + number_reference_sentences
print ("Number of references: ", total_references)
print ("Number of sentences from reference text: ",total_number_reference_sentences)
print ("Average number of sentences per reference text: ", total_number_reference_sentences/total_references)
print ("Number of citances that have more than 1 sentence: ", count_multiple_references_sentences)

print ("\n")

#2019 data
print ("2019 DATA ANALYSIS: ")
citing_text_2019 = all_2019_data["citance_text"]
total_citances = len(citing_text_2019)
total_number_citing_sentences = 0
count_multiple_citing_sentences = 0
for citing_text in citing_text_2019:
#     text_only = re.search('>(.*)</S', citing_text)
#     print (text_only.group(1) + "\n")
#     print (citing_text + "\n")
    number_citing_sentences = citing_text.count("</S>")
    if number_citing_sentences >=2:
        count_multiple_citing_sentences += 1
    total_number_citing_sentences = total_number_citing_sentences + number_citing_sentences
print ("Number of citances: ", total_citances)
print ("Number of sentences from citances: ",total_number_citing_sentences)
print ("Average number of sentences per citance: ", total_number_citing_sentences/total_citances)
print ("Number of citances that have more than 1 sentence: ", count_multiple_citing_sentences)
    
reference_text_2019 = all_2019_data["reference_text"]
total_references = len(reference_text_2019)
total_number_reference_sentences = 0
count_multiple_references_sentences = 0
for reference_text in reference_text_2019:
#     text_only = re.search('>(.*)</S', citing_text)
#     print (text_only.group(1) + "\n")
#     print (citing_text + "\n")
    number_reference_sentences = reference_text.count("</S>")
    if (number_reference_sentences>=2):
        count_multiple_references_sentences += 1
    total_number_reference_sentences = total_number_reference_sentences + number_reference_sentences
print ("Number of references: ", total_references)
print ("Number of sentences from reference text: ",total_number_reference_sentences)
print ("Average number of sentences per reference text: ", total_number_reference_sentences/total_references)
print ("Number of citances that have more than 1 sentence: ", count_multiple_references_sentences)

    

2018 DATA ANALYSIS: 
Number of citances:  753
Number of sentences from citances:  996
Average number of sentences per citance:  1.3227091633466135
Number of citances that have more than 1 sentence:  179
Number of references:  753
Number of sentences from reference text:  1149
Average number of sentences per reference text:  1.5258964143426295
Number of citances that have more than 1 sentence:  265


2019 DATA ANALYSIS: 
Number of citances:  16589
Number of sentences from citances:  16586
Average number of sentences per citance:  0.9998191572728917
Number of citances that have more than 1 sentence:  0
Number of references:  16589
Number of sentences from reference text:  33140
Average number of sentences per reference text:  1.9977093254566278
Number of citances that have more than 1 sentence:  16558


In [4]:
# DISCOURSE FACET OF MANUAL DATA 2018
discourse_facet_2018 = all_2018_data["discourse_facet"]
print ("Total number of citances: ", len(discourse_facet_2018))
facet_dict = {}
for facet in discourse_facet_2018:
    facet = facet.lstrip()
    facet = facet.strip('][').replace("'","").split(',') 
    for subfacet in facet:
        subfacet = subfacet.lstrip()
        if subfacet not in facet_dict:
            facet_dict[subfacet] = 1
        else:
            facet_dict[subfacet] += 1
for key, value in facet_dict.items():
    print (key, value)
                   

Total number of citances:  753
Method_Citation 540
Result_Citation 4
Results_Citation 100
Implication_Citation 60
Aim_Citation 61
Method_CItation 1
Hypothesis_Citation 18
Aim_citation 1
Method Citation 11
Result Citation 1
Method citation 12
